In [107]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

from tqdm import tqdm
import time
from src.scraper import main

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
input_keywords = urllib.parse.quote(input("Enter skill/s: "))
input_location = urllib.parse.quote(input("Enter location: "))

base_url = f"https://ph.indeed.com/jobs?q={input_keywords}&l={input_location}"
delay = 2


ids = []
titles = []
companies = []
locations = []
descriptions = []
urls=[]

for idx in range(0, 1000, 10):
    # print(idx)
    curr_url = f"{base_url}&start={idx}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    }

    response = requests.get(url=base_url, headers=headers)
    # print(f"curr_url: {curr_url} returned {response.status_code}")    # debugger

    if response.status_code == 200:
        time.sleep(delay)   # 2 seconds delay
        
        soup = BeautifulSoup(response.content, "html.parser")
        a_elements = soup.find_all('a')
        _ids = [element.get("id") for element in a_elements if element.get("id") and element.get("id").startswith("sj_")]

        for _id in tqdm(_ids):
            url = f"{base_url}&vjk={_id.split('_')[1]}"
            response = requests.get(url=url, headers=headers)

            if response.status_code == 200:
                soup_inner = BeautifulSoup(response.content, "html.parser")

                job_titles = soup_inner.find_all("h2", attrs={"data-testid": "jobsearch-JobInfoHeader-title"})
                job_company = soup_inner.find_all("a", class_="css-1ioi40n e19afand0")
                job_locations = soup_inner.find_all("div", id="jobLocationText")
                job_description = soup_inner.find_all("div", id="jobDescriptionText")

                for title, company, location, description in zip(job_titles, job_company, job_locations, job_description):
                    ids.append(_id)
                    titles.append(title.text.strip())
                    companies.append(company.text.strip())
                    locations.append(location.text.strip())
                    
                    child_elements = description.find_all(recursive=False)
    
                    # Extract text from each child element and join them together
                    description_text = ' '.join(element.get_text(separator=' ', strip=True) for element in child_elements)
                    descriptions.append(description_text)
                    
                    urls.append(url)

100%|██████████| 7/7 [00:04<00:00,  1.41it/s]


In [113]:
jobs_listing = pd.DataFrame({
    "id": ids,
    "title": titles,
    "company": companies,
    "location": locations,
    "description": descriptions,
    "url": urls
})

jobs_listing = jobs_listing.drop_duplicates(subset="id")
jobs_listing = jobs_listing.set_index("id")
jobs_listing

,title,company,location,description,url
id,,,,,
sj_6092e087391b9f6c,Data Analyst Team Leader - job post,Playmate Leisure Solutions Corp,Taguig,About Us: Playmate Leisure Solutions Corp. is ...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_c87f6edd0b229722,Reports Analyst - job post,Enshored,Pasig,ESSENTIAL DUTIES AND RESPONSIBILITIES You will...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_de6660f6e8faee67,Data Analyst - job post,Visaya Knowledge Process Outsourcing Corporation,Pasig,Responsibilities: Creation of dashboards Gener...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_253f29858cd43c5d,Power BI Specialist - job post,"JKC Outsourcing Solutions, Inc.",Manila,"Position Summary: Responsible for designing, d...",https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_f9947544b305fff6,Data Analyst(s) / Engineer(s) - job post,Credit Corp Group Limited,Makati,We are an Australian owned top 200 ASX listed ...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_22b3e568add35c48,Payment Lifecycle Analyst - Investigations - j...,JPMorgan Chase & Co,Manila,JOB DESCRIPTION We are looking for a passionat...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_67fe7b593107f8fe,Reporting Senior Analyst - job post,Accenture Inc.,Manila,APPLICATION REMINDER: We appreciate your inter...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_b8e0991393a4c9dc,Research Analyst - job post,Hakuhodo/BCI,Pasig,• Help develop strategic omnichannel plans tha...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_a3190dadf299d2e2,Market Research Analyst - job post,Casa Bella Home and Living Corp,Pasig,Market Researcher Analyst - Conducts market re...,https://ph.indeed.com/jobs?q=data%20analyst&l=...


In [114]:
# TODO: Test whether this works. Tailor to the study, focus on OSY

import requests

url = "https://linkedin-data-scraper.p.rapidapi.com/search_jobs"

payload = {
	"keywords": "Data Science",
	"location": "Quezon City",
	"count": 50
}
headers = {
	"content-type": "application/json",
	"X-RapidAPI-Key": "aacc989fcbmsh273b7841567d07cp15970djsn04a3ea8bdf91",
	"X-RapidAPI-Host": "linkedin-data-scraper.p.rapidapi.com"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

25